# モデル蒸留を活用したモデルのファインチューニング

OpenAIは最近**蒸留（Distillation）**をリリースしました。これにより、（大きな）モデルの出力を活用して別の（小さな）モデルをファインチューニングできるようになりました。小さなモデルに移行することで、特定のタスクにおけるコストとレイテンシを大幅に削減できます。このクックブックでは、データセットを確認し、gpt-4oの出力をgpt-4o-miniに蒸留し、汎用的で蒸留されていない4o-miniよりも大幅に良い結果を得る方法を示します。

また、enumのリストを使用した分類問題に対して**構造化出力（Structured Outputs）**も活用します。ファインチューニングされたモデルが構造化出力からどのような恩恵を受けるか、そしてそれがパフォーマンスにどのような影響を与えるかを確認します。蒸留されたモデルを含む、これらすべてのモデルで**構造化出力**が機能することを示します。

まず、データセットを分析し、4oと4o miniの両方の出力を取得して、両モデルのパフォーマンスの違いを明らかにし、その後蒸留を実行してこの蒸留されたモデルのパフォーマンスを分析します。

## 前提条件

依存関係をインストールして読み込みましょう。
OpenAI APIキーが環境変数として"OPENAI_API_KEY"で定義されていることを確認してください。これによりクライアントが直接読み込むことができます。

In [8]:
! pip install openai tiktoken numpy pandas tqdm --quiet

In [9]:
import openai
import json
import tiktoken
from tqdm import tqdm
from openai import OpenAI
import numpy as np
import concurrent.futures
import pandas as pd

client = OpenAI()

## データセットの読み込みと理解

このクックブックでは、以下のKaggleチャレンジからデータを読み込みます：[https://www.kaggle.com/datasets/zynicide/wine-reviews](https://www.kaggle.com/datasets/zynicide/wine-reviews)。

このデータセットは非常に多くの行を持っており、全データでこのクックブックを実行することも可能ですが、偏見のあるフランスワイン愛好家として、行数とブドウ品種を絞り込むために、データセットをフランスワインのみに限定します。

私たちは分類問題に取り組んでおり、説明文、サブリージョン、プロンプトに含める予定の州など、利用可能な他のすべての基準に基づいてブドウ品種を推測したいと考えています。これによりモデルに多くの情報が提供されますが、生産地域など、モデルに大幅に役立つ可能性のある一部の情報を削除して、ブドウ品種の発見において良い仕事をするかどうかを確認することも自由にできます。

レビューで5回未満しか出現しないブドウ品種をフィルタリングしましょう。

このデータセットから500行のランダムなサブセットで進めましょう。

In [10]:
df = pd.read_csv('data/winemag/winemag-data-130k-v2.csv')
df_france = df[df['country'] == 'France']

# Let's also filter out wines that have less than 5 references with their grape variety – even though we'd like to find those
# they're outliers that we don't want to optimize for that would make our enum list be too long
# and they could also add noise for the rest of the dataset on which we'd like to guess, eventually reducing our accuracy.

varieties_less_than_five_list = df_france['variety'].value_counts()[df_france['variety'].value_counts() < 5].index.tolist()
df_france = df_france[~df_france['variety'].isin(varieties_less_than_five_list)]

df_france_subset = df_france.sample(n=500)
df_france_subset.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
95206,95206,France,"Full, fat, ripe, perfumed wine that is full of...",Château de Mercey Premier Cru,91,35.0,Burgundy,Mercurey,NaN,Roger Voss,@vossroger,Antonin Rodet 2010 Château de Mercey Premier C...,Pinot Noir,Antonin Rodet
66403,66403,France,"For simple Chablis, this is impressive, rich, ...",Domaine,89,26.0,Burgundy,Chablis,NaN,Roger Voss,@vossroger,William Fèvre 2005 Domaine (Chablis),Chardonnay,William Fèvre
71277,71277,France,This 50-50 blend of Marselan and Merlot opens ...,La Remise,84,13.0,France Other,Vin de France,NaN,Lauren Buzzeo,@laurbuzz,Domaine de la Mordorée 2014 La Remise Red (Vin...,Red Blend,Domaine de la Mordorée
27484,27484,France,The medium-intense nose of this solid and easy...,Authentic & Chic,86,10.0,France Other,Vin de France,NaN,Lauren Buzzeo,@laurbuzz,Romantic 2014 Authentic & Chic Cabernet Sauvig...,Cabernet Sauvignon,Romantic
124917,124917,France,"Fresh, pure notes of Conference pear peel enti...",NaN,89,30.0,Alsace,Alsace,NaN,Anne Krebiehl MW,@AnneInVino,Domaine Vincent Stoeffler 2015 Pinot Gris (Als...,Pinot Gris,Domaine Vincent Stoeffler


すべてのブドウ品種を取得して、プロンプトと構造化出力のenum リストに含めましょう。

In [11]:
varieties = np.array(df_france['variety'].unique()).astype('str')
varieties

array(['Gewürztraminer', 'Pinot Gris', 'Gamay',
       'Bordeaux-style White Blend', 'Champagne Blend', 'Chardonnay',
       'Petit Manseng', 'Riesling', 'White Blend', 'Pinot Blanc',
       'Alsace white blend', 'Bordeaux-style Red Blend', 'Malbec',
       'Tannat-Cabernet', 'Rhône-style Red Blend', 'Ugni Blanc-Colombard',
       'Savagnin', 'Pinot Noir', 'Rosé', 'Melon',
       'Rhône-style White Blend', 'Pinot Noir-Gamay', 'Colombard',
       'Chenin Blanc', 'Sylvaner', 'Sauvignon Blanc', 'Red Blend',
       'Chenin Blanc-Chardonnay', 'Cabernet Sauvignon', 'Cabernet Franc',
       'Syrah', 'Sparkling Blend', 'Duras', 'Provence red blend',
       'Tannat', 'Merlot', 'Malbec-Merlot', 'Chardonnay-Viognier',
       'Cabernet Franc-Cabernet Sauvignon', 'Muscat', 'Viognier',
       'Picpoul', 'Altesse', 'Provence white blend', 'Mondeuse',
       'Grenache-Syrah', 'G-S-M', 'Pinot Meunier', 'Cabernet-Syrah',
       'Vermentino', 'Marsanne', 'Colombard-Sauvignon Blanc',
       'Gros and Peti

## プロンプトの生成

プロンプトを生成する関数を構築し、リストの最初のワインで試してみましょう。

In [12]:
def generate_prompt(row, varieties):
    # Format the varieties list as a comma-separated string
    variety_list = ', '.join(varieties)
    
    prompt = f"""
    Based on this wine review, guess the grape variety:
    This wine is produced by {row['winery']} in the {row['province']} region of {row['country']}.
    It was grown in {row['region_1']}. It is described as: "{row['description']}".
    The wine has been reviewed by {row['taster_name']} and received {row['points']} points.
    The price is {row['price']}.

    Here is a list of possible grape varieties to choose from: {variety_list}.
    
    What is the likely grape variety? Answer only with the grape variety name or blend from the list.
    """
    return prompt

# Example usage with a specific row
prompt = generate_prompt(df_france.iloc[0], varieties)
prompt

'\n    Based on this wine review, guess the grape variety:\n    This wine is produced by Trimbach in the Alsace region of France.\n    It was grown in Alsace. It is described as: "This dry and restrained wine offers spice in profusion. Balanced with acidity and a firm texture, it\'s very much for food.".\n    The wine has been reviewed by Roger Voss and received 87 points.\n    The price is 24.0.\n\n    Here is a list of possible grape varieties to choose from: Gewürztraminer, Pinot Gris, Gamay, Bordeaux-style White Blend, Champagne Blend, Chardonnay, Petit Manseng, Riesling, White Blend, Pinot Blanc, Alsace white blend, Bordeaux-style Red Blend, Malbec, Tannat-Cabernet, Rhône-style Red Blend, Ugni Blanc-Colombard, Savagnin, Pinot Noir, Rosé, Melon, Rhône-style White Blend, Pinot Noir-Gamay, Colombard, Chenin Blanc, Sylvaner, Sauvignon Blanc, Red Blend, Chenin Blanc-Chardonnay, Cabernet Sauvignon, Cabernet Franc, Syrah, Sparkling Blend, Duras, Provence red blend, Tannat, Merlot, Malbec

クエリを実行する前にコストを把握するために、tiktokenを活用して送信するトークン数とそれに関連する実行コストを理解することができます。これは補完の実行に対する推定値のみを提供し、ファインチューニングプロセス（このクックブックの後半で蒸留を実行する際に使用）のコストは含まれません。ファインチューニングのコストは、エポック数やトレーニングセットなど、他の要因に依存します。

In [13]:
# Load encoding for the GPT-4 model
enc = tiktoken.encoding_for_model("gpt-4o")

# Initialize a variable to store the total number of tokens
total_tokens = 0

for index, row in df_france_subset.iterrows():
    prompt = generate_prompt(row, varieties)
    
    # Tokenize the input text and count tokens
    tokens = enc.encode(prompt)
    token_count = len(tokens)
    
    # Add the token count to the total
    total_tokens += token_count

print(f"Total number of tokens in the dataset: {total_tokens}")
print(f"Total number of prompts: {len(df_france_subset)}")

Total number of tokens in the dataset: 245439
Total number of prompts: 500


In [14]:
# outputing cost in $ as of 2024/10/16

gpt4o_token_price = 2.50 / 1_000_000  # $2.50 per 1M tokens
gpt4o_mini_token_price = 0.150 / 1_000_000  # $0.15 per 1M tokens

total_gpt4o_cost = gpt4o_token_price*total_tokens
total_gpt4o_mini_cost = gpt4o_mini_token_price*total_tokens

print(total_gpt4o_cost)
print(total_gpt4o_mini_cost)

0.6135975
0.03681585


## 完了結果を保存するための関数の準備

限定されたレスポンスのリスト（ブドウ品種の列挙リスト）を扱っているため、構造化出力を活用して、モデルがこのリストから確実に回答するようにしましょう。これにより、モデルの回答をブドウ品種と直接比較し、決定論的な回答を得ることができます（「そのブドウはピノ・ノワールだと思います」ではなく、単に「ピノ・ノワール」と回答するモデルと比較して）。さらに、データセットにないブドウ品種を避けることで、パフォーマンスの向上も図れます。

構造化出力について詳しく知りたい場合は、この[クックブック](https://cookbook.openai.com/examples/structured_outputs_intro)とこの[ドキュメントガイド](https://platform.openai.com/docs/guides/structured-outputs/introduction)をお読みください。

In [15]:
response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "grape-variety",
        "schema": {
            "type": "object",
            "properties": {
                "variety": {
                    "type": "string",
                    "enum": varieties.tolist()
                }
            },
            "additionalProperties": False,
            "required": ["variety"],
        },
        "strict": True
    }
}

モデルを蒸留するには、モデルからのすべての補完を保存する必要があり、これにより小さなモデルをファインチューニングする際の参考として提供できます。そのため、gpt-4oからの補完を保存できるよう、`client.chat.completions.create`メソッドに`store=True`パラメータを追加しています。

すべての補完（4o-miniや将来のファインチューニングされたモデルも含む）を保存することで、OpenAIプラットフォームから直接[Evals](https://platform.openai.com/docs/guides/evals)を実行できるようになります。

これらの補完を保存する際は、メタデータタグと一緒に保存すると便利です。これにより、OpenAIプラットフォームからフィルタリングして、実行したい特定の補完セットに対して蒸留とEvalsを実行できるようになります。

In [16]:
# Initialize the progress index
metadata_value = "wine-distillation" # that's a funny metadata tag :-)

# Function to call the API and process the result for a single model (blocking call in this case)
def call_model(model, prompt):
    response = client.chat.completions.create(
        model=model,
        store=True,
        metadata={
            "distillation": metadata_value,
        },
        messages=[
            {
                "role": "system",
                "content": "You're a sommelier expert and you know everything about wine. You answer precisely with the name of the variety/blend."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
         response_format=response_format
    )
    return json.loads(response.choices[0].message.content.strip())['variety']

## 並列処理

大量の行に対してこれを実行するため、これらの補完を並列で実行し、concurrent futuresを使用するようにしましょう。データフレームを反復処理し、20行ごとに進捗を出力します。補完を実行するモデルからの補完結果を、`{model}-variety`という列名を使用して同じデータフレームに保存します。

In [17]:
def process_example(index, row, model, df, progress_bar):
    global progress_index

    try:
        # Generate the prompt using the row
        prompt = generate_prompt(row, varieties)

        df.at[index, model + "-variety"] = call_model(model, prompt)
        
        # Update the progress bar
        progress_bar.update(1)
        
        progress_index += 1
    except Exception as e:
        print(f"Error processing model {model}: {str(e)}")

def process_dataframe(df, model):
    global progress_index
    progress_index = 1  # Reset progress index

    # Create a tqdm progress bar
    with tqdm(total=len(df), desc="Processing rows") as progress_bar:
        # Process each example concurrently using ThreadPoolExecutor
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = {executor.submit(process_example, index, row, model, df, progress_bar): index for index, row in df.iterrows()}
            
            for future in concurrent.futures.as_completed(futures):
                try:
                    future.result()  # Wait for each example to be processed
                except Exception as e:
                    print(f"Error processing example: {str(e)}")

    return df

データフレーム全体を処理する前に、call model関数を試してみて出力を確認しましょう。

In [18]:
answer = call_model('gpt-4o', generate_prompt(df_france_subset.iloc[0], varieties))
answer

'Pinot Noir'

素晴らしい！出力としてブドウの品種を取得できることを確認しました。それでは、`gpt-4o`と`gpt-4o-mini`の両方でデータセットを処理し、結果を比較してみましょう。

In [19]:
df_france_subset = process_dataframe(df_france_subset, "gpt-4o")

Processing rows: 100%|███████████████████████████████████████████████| 500/500 [00:41<00:00, 12.09it/s]


In [20]:
df_france_subset = process_dataframe(df_france_subset, "gpt-4o-mini")

Processing rows: 100%|███████████████████████████████████████████████| 500/500 [01:31<00:00,  5.45it/s]


## gpt-4oとgpt-4o-miniの比較

これで、これら2つのモデルのすべてのチャット補完が得られました。期待されるブドウ品種と比較して、それを見つける精度を評価してみましょう。ここでは、実行する文字列チェックが単純なので、Pythonで直接これを行いますが、タスクがより複雑な評価を含む場合は、OpenAI Evalsまたはオープンソースのevalフレームワークを活用できます。

In [21]:
models = ['gpt-4o', 'gpt-4o-mini']

def get_accuracy(model, df):
    return np.mean(df['variety'] == df[model + '-variety'])

for model in models:
    print(f"{model} accuracy: {get_accuracy(model, df_france_subset) * 100:.2f}%")

gpt-4o accuracy: 81.80%
gpt-4o-mini accuracy: 69.00%


gpt-4oがgpt-4o-miniよりもブドウの品種を見つけるのが得意であることがわかります（12.80%高く、4o-miniと比較して相対的に約20%も！）。今、私たちはgpt-4oにトレーニング中にワインを飲ませているのではないかと疑っています！

## gpt-4oの出力をgpt-4o-miniに蒸留する

この予測を頻繁に実行したいと仮定しましょう。完了をより高速で安価にしたいが、その精度レベルは維持したいとします。4oの精度を4o-miniに蒸留できれば素晴らしいですよね？やってみましょう！

OpenAIのStored completionsページに移動します：[https://platform.openai.com/chat-completions](https://platform.openai.com/chat-completions)。

モデルgpt-4oを選択しましょう（これを確実に行ってください。実行した4o-miniの出力を蒸留したくはありません）。また、メタデータ`distillation: wine-distillation`を選択して、このクックブックから実行されたstored completionsのみを取得しましょう。

![Filtering out completions](../images/filtering-out-completions.png)

completionsを選択したら、右上角の「Distill」をクリックして、これらのcompletionsに基づいてモデルをファインチューニングできます。これを行うと、ファインチューニングプロセスを実行するためのファイルが自動的に作成されます。次に、ベースモデルとして`gpt-4o-mini`を選択し、デフォルトパラメータを保持します（ただし、パフォーマンスを向上させるために自由に変更したり反復したりできます）。

![Distilling modal](../images/distilling.png)

ファインチューニングジョブが開始されたら、ファインチューニングページからファインチューニングジョブIDを取得できます。これを使用してファインチューニングジョブのステータスを監視し、完了後にファインチューニングされたモデルIDを取得します。

![Fine tuning job](../images/fine-tuning-job.png)

In [22]:
# copy paste your fine-tune job ID below
finetune_job = client.fine_tuning.jobs.retrieve("ftjob-pRyNWzUItmHpxmJ1TX7FOaWe")

if finetune_job.status == 'succeeded':
    fine_tuned_model = finetune_job.fine_tuned_model
    print('finetuned model: ' + fine_tuned_model)
else:
    print('finetuned job status: ' + finetune_job.status)

finetuned model: ft:gpt-4o-mini-2024-07-18:distillation-test:wine-distillation:AIZntSyE


## 蒸留モデルでの補完実行

モデルのファインチューニングが完了したので、このモデルを使用して補完を実行し、gpt4oとgpt4o-miniの両方と精度を比較できます。
フランスワインの異なるサブセットを取得しましょう（出力をフランスのブドウ品種に制限し、外れ値を除外したため、検証データセットもこれに焦点を当てる必要があります）。各モデルで300エントリに対してこれを実行しましょう。

In [23]:
validation_dataset = df_france.sample(n=300)

models.append(fine_tuned_model)

for model in models:
    another_subset = process_dataframe(validation_dataset, model)

Processing rows: 100%|███████████████████████████████████████████████| 300/300 [00:37<00:00,  8.08it/s]


モデルの精度を比較してみましょう

In [24]:
for model in models:
    print(f"{model} accuracy: {get_accuracy(model, another_subset) * 100:.2f}%")

gpt-4o accuracy: 79.67%
gpt-4o-mini accuracy: 64.67%
ft:gpt-4o-mini-2024-07-18:distillation-test:wine-distillation:AIZntSyE accuracy: 79.33%


これは非蒸留のgpt-4o-miniと比較して**約22%の相対的な改善です！🎉**

私たちのファインチューニングされたモデルは、同じベースモデルを持ちながら、gpt-4o-miniよりもはるかに優れた性能を発揮します。このモデルを使用することで、将来のブドウ品種予測において、より低コストかつ低レイテンシで推論を実行できるようになります。